# Deliverable 3. Create a Travel Itinerary Map

## Make sure the initial dependencies and the Google Maps API key are imported

In [50]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
%store -r gg_key

In [51]:
# Configure gmaps
gmaps.configure(api_key=gg_key)

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [52]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("./Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Kiama,AU,81.37,clear sky,-34.6833,150.8667,The Sebel Kiama Harbourside
1,4,Rikitea,PF,78.01,light rain,-23.1203,-134.9692,People ThankYou
2,5,Buala,SB,81.93,moderate rain,-8.1450,159.5921,Maringe Lagoon Lodge
3,8,Port Alfred,ZA,74.17,scattered clouds,-33.5906,26.8910,The Halyards Hotel & Spa
4,9,Komatipoort,ZA,80.06,scattered clouds,-25.4332,31.9548,Kruger View Backpackers


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [53]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]



In [54]:
# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# Display the figure
fig




Figure(layout=FigureLayout(height='420px'))

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [55]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"] == "Nelson Bay"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Nelson Bay"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Kiama"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Ulladulla"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Batemans Bay"]

In [56]:
# Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start["Lat"].to_numpy()[0],vacation_start["Lng"].to_numpy()[0]
end = vacation_end["Lat"].to_numpy()[0],vacation_end["Lng"].to_numpy()[0]
stop1 = vacation_stop1["Lat"].to_numpy()[0],vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0],vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0],vacation_stop3["Lng"].to_numpy()[0]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [57]:

# Define a new figure object
fig = gmaps.figure()

# Create a direction layer map using the start and end latitude-longitude pairs, and stop1, stop2, and stop3 as the waypoints.
# The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
vacation_itinerary = gmaps.directions_layer(
    start,end,waypoints=[stop1,stop2,stop3],
    travel_mode='DRIVING')

# Add the layer to the map
fig.add_layer(vacation_itinerary)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))

## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [58]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = pd.concat(
    [
        vacation_start,
        vacation_stop1,
        vacation_stop2,
        vacation_stop3
    ],
    ignore_index = True
)


# Display sample data
waypoints_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,490,Nelson Bay,AU,84.25,clear sky,-32.7167,152.1500,Mantra Nelson Bay
1,0,Kiama,AU,81.37,clear sky,-34.6833,150.8667,The Sebel Kiama Harbourside
2,126,Ulladulla,AU,74.07,clear sky,-35.3500,150.4667,Sandpiper Motel
3,398,Batemans Bay,AU,72.10,clear sky,-35.7167,150.1833,Batemans Bay Marina Resort


## Use GeoViews to create map that shows the four cities in the itinerary

In [59]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in waypoints_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = waypoints_df[["Lat","Lng"]]

locations.head()

,Lat,Lng
0,-32.7167,152.1500
1,-34.6833,150.8667
2,-35.3500,150.4667
3,-35.7167,150.1833


In [60]:
# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Define a new figure object
fig = gmaps.figure(center=(53.0, 0.0), zoom_level=6)

# Add the layer to the map
heat_layer = gmaps.heatmap_layer(locations, dissipating=False,
             max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Display the map
fig

Figure(layout=FigureLayout(height='420px'))

SyntaxError: invalid syntax (2107536509.py, line 2)